<a href="https://colab.research.google.com/github/ruchira559/chat-with-pdf/blob/main/research_and_prototyping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Libraries
!pip install -q langchain-groq langchain_community
!pip install -q pypdf
!pip install requests==2.32.4

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Attempting uninstall: requests
    Found existing installation: requests 2.32.5
    Uninstalling requests-2.32.5:
      Successfully uninstalled requests-2.32.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.4.1 requires requests<3.0.0,>=2.32.5, but you have requests 2.32.4 which is incompatible.


In [2]:
from google.colab import userdata
from langchain_groq import ChatGroq

try:
    # 1. Retrieve the key you already saved in your Secrets
    groq_api_key = userdata.get('GROQ_API_KEY')

    # 2. Initialize with the updated 2026 model name
    # Using 'llama-3.1-8b-instant' as it is the stable replacement
    llm = ChatGroq(
        temperature=0,
        groq_api_key=groq_api_key,
        model_name="llama-3.1-8b-instant"
    )

    # 3. Test the connection
    response = llm.invoke("Confirm connection: Reply with 'System Ready'.")
    print(f"Status: {response.content}")

except Exception as e:
    print(f"Error encountered: {e}")

Status: System Ready.


In [3]:
# Import necessary libraries
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
# Load the PDF
loader = PyPDFLoader("/content/data.pdf")
data = loader.load()

In [6]:
# Initialize the Splitter
# chunk_size: Max characters in each chunk
# chunk_overlap: Keep a bit of the previous chunk to maintain context
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", "\n", ".", " ", ""]
)

In [7]:
# Perform the split
chunks = text_splitter.split_documents(data)

In [9]:
# Verification Print
print(f"Document loaded: {len(data)} pages found.")
print(f"Document split into {len(chunks)} smaller chunks.")
print("-" * 30)
print(f"Sample from Chunk #1:\n{chunks[0].page_content[:200]}...")

Document loaded: 12 pages found.
Document split into 57 smaller chunks.
------------------------------
Sample from Chunk #1:
Vol.:(0123456789)
The International Journal of Life Cycle Assessment 
https://doi.org/10.1007/s11367-024-02405-8
DATA AVAILABILITY , DATA QUALITY
Testing the use of a large language model (LLM) for pe...
